[:de:](index-de.ipynb) [:us:](index.ipynb)

In [1]:
#|include: false

!sqlite3 fin_de.db .databases .quit

main: /home/git_repos/blog/posts/2022-11-13-repay-vs-invest/fin_de.db r/w


In [2]:
#|include: false
from IPython.display import Markdown as md
import sqlite3
import sqlalchemy
import pandas as pd
import numpy as np

In [3]:
#|include: false

# enable sql magic for Jupyter notebook

# load jupyter extention (requires pip install ipython-sql)
%load_ext sql
# disable autocommit
%config SqlMagic.autocommit=False

In [4]:
#|include: false

%sql sqlite:///fin.db
    
# Test connection
v1 = "Ready to go!"
%sql df_test << SELECT :v1 as "Test"
assert(df_test.DataFrame().values[0]==v1)

 * sqlite:///fin.db
Done.
Returning data to local variable df_test


:::{.callout-warning}
**Disclaimer:** Dieser Blog-Eintrag stellt keine Finanzempfehlung oder -beratung dar! Es ist eine Beispielrechnung. Dieser Blog-Eintrag enthält zahlreiche unrealistische und unvollständige Annahmen. Alle Zahlen sind ausgedacht. Wende Dich an einen professionellen Finanzberater, falls Du Hilfe bei Finanzen brauchst. Und am wichtigsten, schnapp Dir eine Tabellenkalkulation oder ein Blattpapier und rechne eigenständig mit eigenen Annahmen und Zahlen.
:::

# Eine knifflige Frage
Am letzten Wochenende hatte ich eine Diskussion darüber geführt, ob man überschüßiges Geld besser für eine Sondertilg verwendet oder stattdessen das Geld anlegen sollte.

:man:: *Ich habe gerade eine Sondertilgung vorgenommen um meine Hausschulden zu reduzieren.*

:bust_in_silhouette:: *Wie hoch sind denn die Zinsen für den Hauskredit?*

:man:: *2.5%* - nochmal, die Zahlen sind erfunden

:bust_in_silhouette:: *Hast Du nicht darüber nachgedacht das Geld stattdessen zu investieren? Selbst wenn Du Risiko-averse bist, bekämst Du schon 2.8% Zinsen für ein Festgeldkonto?*

:man:: *Ich hatte darüber nachgedacht. Es machte aber wenig Sinn.*

:bust_in_silhouette:: *Aber wieso? Du hättest 0.3% pro Jahr dazu gewonnen.*

Wer liegt richtig in diesem Fall? Die Antwort ist nicht eindeutig.

# Beispieldaten
Bevor wir in die Analyse einsteigen, denken wir uns einige Beispieldaten aus:

In [5]:
principal = 100000 # Hauskredithöhe
interest_rate = 0.025 # Kreditzinsen
m_payment = 5000 # Jährliche Rate um Zinsen zu zahlen und das Haus abzubezahlen
reference_rate = 0.028 # Zinssatz der risikolosen anlage
lump_sum = 15000 # Geld, das als Sondertilgung oder Investment ausgegeben werden kann 
ls_year = 2 # Jahr der Anlage oder Sondertilgung
appreciation = 0.02 #  jährliche Wertsteigerung des Hauses

Zudem treffen wir noch weitere Annahmen, um das Beispiel zu vereinfachen.

- Keine Gebühren, keine Steuern.
- Die jährliche Rate bleibt gleich. 
- Sobald das Haus abgezahlt ist, wird der restliche Betrag der Rate in die risikolose Anlage investiert.
- Zinsen für den Hauskredit und die risikolose Anlage bleiben immer gleich.
- Das Extrageld kann als Investment oder Sondertilgung verwendet werden.
- Das angelegte Geld wird nicht abgehoben, bis das Haus abbezahlt ist.
- Zinserträge werden reinvestiert. 

:::{.callout-important}
Diese Annahmen sind unrealistisch im echten Leben. Ebenfalls kann es bei Änderung der Beispieldaten zu anderen Schlussfolgerungen kommen, als diese hier beschrieben sind.
:::

# Der einfache Fall

Zuerst schauen wir uns die Gedanken von :bust_in_silhouette: etwas genauer an:


In [6]:
#|echo: false

md(f"""
Eine Sondertilgung reduziert meinen jährlichen Zinsaufwand um {interest_rate*100:.2f}%. 
Das stellt einen jährlichen Wertegewinn von {interest_rate*100:.2f}% dar.\n
Falls das Geld alternativ in Festgeld investiert würde, liese es sich mit {reference_rate*100:.2f}% verzinsen. 
Das entspricht einen Wertezuwachs von {reference_rate*100:.2f}% pro Jahr.

```Hauszins - Anlagezins = {interest_rate*100:.2f}% - {reference_rate*100:.2f}% = {(interest_rate - reference_rate)*100:.2f}%```

Die Anlage hätte eine {(reference_rate - interest_rate)*100:.2f}% höhere Rendite im Vergleich zur Sondertilgung.\n
Beide Szenarion profitieren vom Zinseszinseffekt, wie man an diesem Beispiel erkennen kann:
""")




Eine Sondertilgung reduziert meinen jährlichen Zinsaufwand um 2.50%. 
Das stellt einen jährlichen Wertegewinn von 2.50% dar.

Falls das Geld alternativ in Festgeld investiert würde, liese es sich mit 2.80% Zinsen sparen. 
Das entspricht einen Wertezuwachs von 2.80% pro Jahr.

```Hauszins - Anlagezuns = 2.50% - 2.80% = -0.30%```

Die Anlage hätte eine 0.30% höhere Rendite im Vergleich zur Sondertilgung.

Beide Szenarion profitieren vom Zinseszinseffekt, wie man an diesem Beispiel erkennen kann:


In [7]:
#|include: false

%%sql simple_calc <<
with plan(year, payment, zinsen, opp_investment) as
(
    SELECT 0 AS year,
           0 AS payment,
           0 AS zinsen,
           0 AS opp_investment
    UNION
    SELECT p.year + 1 AS year,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS payment,
           p.zinsen * (1+:interest_rate) +
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS zinsen,
           p.zinsen * (1+:reference_rate) +
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END AS opp_investment
      FROM plan p
     WHERE p.year < 7
)
SELECT year as year,
       payment as payment,
       max(round(zinsen)-:lump_sum, 0) AS savings_of_mortage_interest, 
       max(round(opp_investment)-:lump_sum, 0) AS gain_from_unrisky_investment,
       abs(max(round(zinsen)-:lump_sum, 0) - 
       max(round(opp_investment)-:lump_sum, 0)) AS diff
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable simple_calc


In [8]:
#|echo: false

simple_calc_df = simple_calc.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Jahr', axis=1)
simple_calc_df.columns = ['Zahlung', 'Gesparte Zinsen bei Sondertilgung', 'Zinserträge bei Investment', 'Differenze']
simple_calc_df

,Zahlung,Gesparte Zinsen bei Sondertilgung,Zinserträge bei Investment,Differenze
0,0,0.0,0.0,0.0
1,0,0.0,0.0,0.0
2,15000,0.0,0.0,0.0
3,0,375.0,420.0,45.0
4,0,759.0,805.0,46.0
5,0,1153.0,1201.0,48.0
6,0,1557.0,1606.0,49.0
7,0,1971.0,2021.0,50.0


In [9]:
#|echo: false

md(f"""In diesem Beispiel haben wir einen erhöhten Wertzuwachs von {simple_calc_df['Differenze'].values[-1:][0]} € 
nach fünf Jahren wenn wir das Geld anlegen statt sonderzutilgen.""")

In diesem Beispiel haben wir einen erhöhten Wertzuwachs von 50.0 € 
nach fünf Jahren wenn wir das Geld anlegen statt sonderzutilgen.

# Tilgungsplan
Schauen wir uns die Zahlen etwas genauer an und erstellen einen Tilgungsplan für die gesamte Tilgungsphase.

In [10]:
#|include: false

%%sql amor_plan <<
with plan(year, interest, principal, balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           :principal AS balance
    UNION
    SELECT p.year + 1 AS year,
           -p.balance * :interest_rate AS interest,
           MIN(:m_payment - p.balance * :interest_rate, p.balance) AS principal,
           p.balance - (min(:m_payment - p.balance * :interest_rate, p.balance) /*principal*/ ) AS balance
      FROM plan p
     WHERE p.balance - (min(:m_payment - p.balance * :interest_rate, p.balance) + 0) /*Restschuld*/ >= 0 
       AND MIN(:m_payment - p.balance * :interest_rate, p.balance) /*principal*/ > 0 
)
SELECT year,
       interest AS interest, 
       principal AS principal,  
       -balance AS balance
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable amor_plan


In [11]:
#|echo: false
amor_plan_df = amor_plan.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Jahr', axis=1)
amor_plan_df.loc['Total:'] = amor_plan_df.sum()
amor_plan_df.loc['Total:',['balance']]=amor_plan_df[['balance']].max() 
amor_plan_df.columns = ['Zinsen', 'Tilgung', 'Restschuld']

vmin = -interest_rate*principal
vmax = m_payment-interest_rate*principal

(amor_plan_df.style
            .format('{:.0f}', na_rep="")
            .bar(subset=pd.IndexSlice[amor_plan_df[amor_plan_df['Zinsen']<=0].index,'Zinsen'],
                     align='right', vmin=vmin, vmax=0, cmap="autumn", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=pd.IndexSlice[amor_plan_df[amor_plan_df['Tilgung']>=0].index,'Tilgung'],
                     align='left', vmin=0, vmax=m_payment, cmap="summer_r", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
            #.bar(subset=['interest'], align='right', vmin=vmin, vmax=0, cmap="autumn",
            #     height=80, width=100, props="width: 100px; border-right: 1px solid gray; border-left: 1px solid gray;"
            #    )
            #.bar(subset=['principal'], align='left', vmin=vmin, vmax=m_payment, cmap="summer_r", 
            #     height=80, width=100, props="width: 100px; border-right: 1px solid gray;"
            #    )
            .bar(subset=['Restschuld'], align='right', vmin=-2.5*principal, vmax=0, cmap="PuRd_r", 
                 height=80, width=80, props="width: 80px; border-right: 1px solid gray;"
                )
                .set_table_styles({
                   ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                   }, overwrite=False, axis=1)
               .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                                  {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                                  {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                                  {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                                  {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                                  {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray;'}
                                 ], overwrite=False)
)

,Zinsen,Tilgung,Restschuld
0,,,-100000
1,-2500,2500,-97500
2,-2438,2562,-94938
3,-2373,2627,-92311
4,-2308,2692,-89619
5,-2240,2760,-86859
6,-2171,2829,-84031
7,-2101,2899,-81131
8,-2028,2972,-78160
9,-1954,3046,-75114


In [12]:
#|echo: false
md(f"Mit einer jährlichen Rate von {m_payment:,.0f} € wäre der Kredit über {principal:,.0f} € nach {amor_plan_df.index.values[-2]:,.0f} Jahren abbezahlt. "+
      f"Die Zinsen für den Kredit beliefen sich auf {-amor_plan_df.loc['Total:','Zinsen']:,.0f} €"
     )

Mit einer jährlichen Rate von 5,000 € wäre der Kredit über 100,000 € nach 29 Jahren abbezahlt. Die Zinsen für den Kredit beliefen sich auf 40,359 €

Der Tilgungsplan veranschaulicht, wie die Zinsen sinken und die Tilgung mit den Jahenen steigt.

# Investition
Erweitern wir den Tilgungsplan um einige Spalten:

- Sondertilgung
- Investition
- Verzinste Investition
- Wertzuwachs der gezahlten Tilgung zum jeweiligen Jahr
- Vermögen (*alle Anlagewerte - Schulden*)
- Vermögen inklusive Wertentwicklung des Hauses

Im ersten Szenario **investieren** wir das Extrageld.

In [13]:
#|include: false

%%sql inv <<
with plan(year, interest, principal, extra_payment, mortgage_balance, investment, investment_balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           NULL AS extra_payment,
           :principal AS mortgage_balance,
           NULL AS investment,
           0 AS investment_balance
    UNION
    SELECT p.year + 1 AS year,
           - p.mortgage_balance * :interest_rate AS interest,
           MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) AS principal,
           0 AS extra_payment,
           p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ +  
                           0 /*extra payment*/) AS mortgage_balance,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END +
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment,
           p.investment_balance*(1+:reference_rate) + 
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 0 END +
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment_balance
      FROM plan p
     WHERE p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ >= 0 
       AND MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ > 0 
)
SELECT year as year, 
       interest AS interest, 
       principal AS principal, 
       extra_payment AS extra_payment,
       -mortgage_balance AS mortgage_balance,
       investment AS investment,
       investment_balance AS investment_balance, 
       sum(principal+extra_payment)over(order by year)*(power(1+:appreciation, year)-1) AS appreciation
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable inv


In [14]:
#|echo: false

def plan_layout(df):
    df = df.DataFrame().set_index('year').reset_index(drop = True).rename_axis('Year', axis=1)
    df.loc['Total:'] = df.sum()
    df.loc['Total:',['mortgage_balance']]=df[['mortgage_balance']].max() #,'investment'
    df.loc['Total:',['appreciation']]=df[['appreciation']].values[-2:][0]
    df.loc['Total:',['investment_balance']]=df['investment_balance'].values[-2:][0]
    df['Net Worth'] = df[['principal','extra_payment']].cumsum().sum(axis=1) + df['investment_balance']
    df.loc['Total:','Net Worth'] = df.loc['Total:'][['principal','extra_payment','mortgage_balance','investment_balance']].sum()
    df['Net Worth (appr.)'] = df['Net Worth']  + df['appreciation']
    df_ret = df.copy()
    
    df.columns = pd.MultiIndex.from_tuples([('Hauskredit', 'Zinsen'),
                                            ('Hauskredit', 'Tilgung'),
                                            ('Hauskredit', 'Sondertilgung'),
                                            ('Hauskredit', 'Restschuld'),
                                            ('Investment', 'Anlagebetrag'),
                                            ('Investment', 'Anlage gesamt'),
                                            ('Wertzuwachs', 'Betrag'),
                                            ('Vermögen', 'ohne Zuwachs'),
                                            ('Vermögen', 'mit Zuwachs')], names=['','Jahr'])
                                            
    vmin = -interest_rate*principal
    vmax = m_payment-interest_rate*principal
    display(df.style
                .format('{:.0f}', na_rep="")
                .bar(subset=pd.IndexSlice[df[df[('Hauskredit', 'Zinsen')]<=0].index,('Hauskredit', 'Zinsen')],
                     align='right', vmin=vmin, vmax=0, cmap="autumn", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=pd.IndexSlice[df[df[('Hauskredit', 'Tilgung')]>=0].index,('Hauskredit', 'Tilgung')],
                     align='left', vmin=0, vmax=m_payment, cmap="summer_r", 
                     height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
                    )
                .bar(subset=[('Hauskredit', 'Sondertilgung')], align='left', vmin=0, vmax=lump_sum, cmap="summer_r", 
                     height=80, width=80, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
                .bar(subset=[('Investment', 'Anlagebetrag')], align='left', vmin=0, vmax=max(lump_sum, m_payment), cmap="summer_r", 
                     height=80, width=80, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                    )
           
               .bar(subset=[('Hauskredit', 'Restschuld')], align='right', vmin=-2.5*principal, vmax=0, cmap="PuRd_r", 
                    height=80, width=100, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                   )
               .bar(subset=[('Wertzuwachs', 'Betrag')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                    height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                   )
               .bar(subset=[('Investment', 'Anlage gesamt')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                    height=80, width=100, props="width: 80px; border-right: 1px solid gray; border-left: 1px solid gray;"
                   )
               .bar(subset=[('Vermögen', 'ohne Zuwachs')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                    height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                   )
               .bar(subset=[('Vermögen', 'mit Zuwachs')], align='left', vmin=0, vmax=2.5*principal, cmap="Blues", 
                    height=80, width=100, props="width: 80px; border-right: 1px solid gray;"
                   )
               .set_table_styles({
                  ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                             {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                  }, overwrite=False, axis=1)
              .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                                 {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                                 {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                                 {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                                 {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                                 {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray;'}
                                ], overwrite=False)
               .set_table_styles({
                       ('Investment', 'Anlage gesamt'): [{'selector': 'th', 'props': 'border-right: 1px solid gray'},
                                  {'selector': 'td', 'props': 'border-right: 1px solid gray'}]
                       }, overwrite=False, axis=0)
    )
    
    return df_ret

inv_df = plan_layout(inv)

In [15]:
#|echo: false
duration = inv_df.index.values[-2]
md(f"Die Laufzeit beträgt {duration} Jahre.")

Die Laufzeit beträgt 29 Jahre.

In der Tilgungsphase verzinst sich die Investition. Zur gleichen Zeit wächst der Wert des Gebäudes an.

# Sondertilgung
Im zweiten Szenario wird das Geld für eine **Sondertilgung** verwendet, um die Restschuld zu reduzieren.

In [16]:
#|include: false

%%sql so <<
with plan(year, interest, principal, extra_payment, mortgage_balance, investment, investment_balance) as
(
    SELECT 0 AS year,
           NULL AS interest,
           NULL AS principal,
           NULL AS extra_payment,
           :principal AS mortgage_balance,
           NULL AS investment,
           0 AS investment_balance
    UNION
    SELECT p.year + 1 AS year,
           - p.mortgage_balance * :interest_rate AS interest,
           MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) AS principal,
           CASE WHEN :ls_year = p.year +1 THEN 
                :lump_sum 
           ELSE 
                0
           END AS extra_payment,
           p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END /*extra payment*/) AS mortgage_balance,
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) + 0) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate 
           ELSE 0 END AS investment,       
           p.investment_balance*(1+:reference_rate) + 
           CASE WHEN p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END) /*mortgage_balance*/ = 0 THEN 
              :m_payment - MIN(:m_payment - p.mortgage_balance * :interest_rate /*principal*/, p.mortgage_balance) - p.mortgage_balance * :interest_rate
           ELSE 0 END AS investment_balance 
      FROM plan p
     WHERE (p.mortgage_balance - (min(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) +  
                           CASE WHEN :ls_year = p.year +1 THEN 
                                :lump_sum 
                           ELSE 
                                0
                           END) /*mortgage_balance*/ >= 0 
       AND MIN(:m_payment - p.mortgage_balance * :interest_rate, p.mortgage_balance) /*principal*/ > 0 
       ) OR p.year + 1 <= :duration
)
SELECT year, 
       interest AS interest, 
       principal AS principal, 
       extra_payment AS extra_payment,
       -mortgage_balance AS mortgage_balance,
       investment AS investment,
       investment_balance AS investment_balance,
       sum(principal+extra_payment)over(order by year)*(power(1+:appreciation, year)-1) AS appreciation
  FROM plan

 * sqlite:///fin.db
Done.
Returning data to local variable so


In [17]:
#|echo: false

so_df = plan_layout(so)

In [18]:
#|echo: false
duration = so_df[so_df['interest']==0].index.values[0]-1
md(f"Die Laufzeit ist jetzt {duration} Jahre.")

Die Laufzeit ist jetzt 23 Jahre.

Der Tilgungsplan zeigt, dass das Haus früher abbezahlt ist. Danach kann die jährliche Rate Investiert werden.

Beim betrachten der *Total*-Zeile fällt auf, dass insgesamt weniger Zinsen für das Haus gezahlt wurden, wenn eine Sondertilgung vorgenommen wurde. Denoch ist das Vermögen höher, wenn stattdessen die Investition vorgenommen worden wäre.

So, :bust_in_silhouette: hatte also recht! Aber ...

# Vergleich der Strategien

Zum Schluss stellen wir das Vermögen der beiden Szenarien nocheinmal gegenüber und berechnen die jährlich Differenz (```<>```).

In [19]:
#|echo: false

compare_df = inv_df[['Net Worth', 'Net Worth (appr.)']].merge(so_df[['Net Worth', 'Net Worth (appr.)']], left_index = True, right_index = True)
compare_df.columns = ['NW invest', 'NW invest (appr.)', 'NW extra pay', 'NW extra pay (appr.)']
compare_df['diff'] = compare_df['NW extra pay'] - compare_df['NW invest']
compare_df['diff (appr.)'] = compare_df['NW extra pay (appr.)'] - compare_df['NW invest (appr.)']

compare_df = compare_df[['NW invest', 'diff', 'NW extra pay', 'NW invest (appr.)', 'diff (appr.)', 'NW extra pay (appr.)']]


vmin=compare_df['diff (appr.)'].min()
vmax=compare_df['diff (appr.)'].max()

compare_df.columns = pd.MultiIndex.from_tuples([('Vermögen (ohne Wertzuwachs)', 'Investment'),
                                                ('Vermögen (ohne Wertzuwachs)', '<>'),
                                                ('Vermögen (ohne Wertzuwachs)', 'Sondertilgung'),
                                                ('Vermögen (mit Wertzuwachs)', 'Investment'),
                                                ('Vermögen (mit Wertzuwachs)', '<>'),
                                                ('Vermögen (mit Wertzuwachs)', 'Sondertilgung')], names=['','Year'])


(compare_df.style
           .format('{:.0f}', na_rep="")
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Vermögen (ohne Wertzuwachs)', '<>')]>=0].index,('Vermögen (ohne Wertzuwachs)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="summer_r", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               )
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Vermögen (ohne Wertzuwachs)', '<>')]<=0].index,('Vermögen (ohne Wertzuwachs)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="autumn", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               ) 
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Vermögen (mit Wertzuwachs)', '<>')]>=0].index,('Vermögen (mit Wertzuwachs)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="summer_r", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               )
           .bar(subset=pd.IndexSlice[compare_df[compare_df[('Vermögen (mit Wertzuwachs)', '<>')]<=0].index,('Vermögen (mit Wertzuwachs)', '<>')],
                align='mid', vmin=vmin, vmax=vmax, cmap="autumn", 
                height=80, width=80, props="width: 100px; border-right: 1px solid gray;"
               ) 
           .set_table_styles({
                   ('Total:'): [{'selector': 'th', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-top: 1px solid gray; border-bottom: 1px solid gray'}]
                   }, overwrite=False, axis=1)
           .set_table_styles([{'selector': 'table', 'props': 'border-spacing: 2px'},
                              {'selector': 'thead', 'props': 'border: 1px solid gray'}, 
                              {'selector': 'th', 'props': 'text-align: center; padding: 4.5px;'},
                              {'selector': 'th.col_heading', 'props': 'border: 1px solid gray'},
                              {'selector': 'tbody', 'props': 'border: 1px solid gray'},
                              {'selector': 'td', 'props': 'text-align: center; border-left: 1px solid gray; border-right: 1px solid gray'}
                             ], overwrite=False)
           .set_table_styles({
                   ('Vermögen (ohne Wertzuwachs)', 'investing'): [{'selector': 'th', 'props': 'border-left: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-left: 1px solid gray'}],
                   ('Vermögen (ohne Wertzuwachs)', 'Sondertilgung'): [{'selector': 'th', 'props': 'border-right: 1px solid gray'},
                              {'selector': 'td', 'props': 'border-right: 1px solid gray'}]
                   }, overwrite=False, axis=0)
)

Wie bereits gesehen ist das Vermögen am Ende des Betrachtungszeitraums beim Investment-Szenario höher. Aber berücksichtigt man die Wertentwicklung des Hauses ist das Bild nicht mehr eindeutig. Würden wir etwa im Sondertilgungsszenario das Haus nach der Abbezahlung (oder schon eher) verkaufen, könnte sich die Sondertilgung lohnen (abhängig von der Höhe des Referenzzinses). 

Übrigens, im Falle eines Wertverlusted das Hauses (negativer Wertzuwachs) hätte die Sondertilgung oder ein vorzeitiger Verkauf einen stark negativen Effekt.

# Schlussfolgerung
Selbst wenn der Sachverhalt einfach erscheint, lohnt es sich immer eine Tabellenkalkulation vorzunehmen und sich verschiedene Szenarien auch grafisch zu betrachten.


# Ressourcen
## Interaktives Notebook
- Mit Zahlen spielen: [![notebook]('../../../../assets/colab.svg?sanitize=true')](https://colab.research.google.com/github/joatom/blog/blob/master/assets/resources/notebooks/2022-11-13-repay-vs-invest.ipynb)
- Notebook downloaden: [![notebook]('../../../../assets/github.svg?sanitize=true')](https://github.com/joatom/blog/blob/master/assets/resources/notebooks/2022-11-13-repay-vs-invest.ipynb)

## Snippets
- [Recursive SQL](../../snippets/2022-11-20-recursive-sql/index.ipynb)
- [Styles in Pandas](../../snippets/2022-11-20-pandas-styles/index.ipynb)